In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
import coremltools as ct

# Define a simple neural network with two layers
class SimpleClassificationModel(nn.Module):
    def __init__(self):
        super(SimpleClassificationModel, self).__init__()
        self.layer1 = nn.Linear(3, 100) # 3 inputs, 10 outputs
        self.layer2 = nn.Linear(100, 5)  # 10 inputs, 10 output

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

# Create the model
model = SimpleClassificationModel()

# Create a sample input tensor
sample_input = torch.rand(1, 3)  # Adjust the shape according to your model's input

# Trace the model with a sample input
traced_model = torch.jit.trace(model, sample_input)

# Convert the traced model to Core ML format
input_features = [ct.TensorType(shape=(1, 3))]
output_features = ["output"]
mlmodel = ct.convert(
    traced_model,
    inputs=input_features,
    convert_to="neuralnetwork"
)
mlmodel.save("classification.mlmodel")


Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 7/7 [00:00<00:00, 4137.56 ops/s]


In [6]:

import coremltools
from coremltools.models.neural_network import NeuralNetworkBuilder, SgdParams, AdamParams
from coremltools.models import datatypes
import numpy as np

spec = coremltools.utils.load_spec('classification.mlmodel')

builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)

builder.inspect_layers(last=6)
builder.inspect_output_features()


[Id: 2], Name: linear_1 (Type: innerProduct)
          Updatable: False
          Input blobs: ['input']
          Output blobs: ['linear_1']
[Id: 1], Name: input (Type: activation)
          Updatable: False
          Input blobs: ['linear_0']
          Output blobs: ['input']
[Id: 0], Name: linear_0 (Type: innerProduct)
          Updatable: False
          Input blobs: ['x']
          Output blobs: ['linear_0']
[Id: 0] Name: linear_1
          Type: multiArrayType {
  dataType: FLOAT32
}



In [ ]:
# Build neural networks using the model specification
builder = NeuralNetworkBuilder(spec=spec)

# Make layers updatable
builder.make_updatable(['linear_0', 'linear_1'])

builder.add_softmax(name='output_prob', input_name='linear_1', output_name='output_prob')
builder.set_categorical_cross_entropy_loss(name='lossLayer', input='output_prob')


# # Manually add a mean squared error loss layer
# feature = ('linear_1', datatypes.Array(1))
# builder.set_mean_squared_error_loss(name='lossLayer', input_feature=feature)


# Define the optimizer (SGD in this example)
# sgd_params = SgdParams(lr=0.001, batch=16)  # Adjust learning rate and batch size as needed
# builder.set_sgd_optimizer(sgd_params)

# define the optimizer (Adam in this example)
adam_params = AdamParams(lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8, batch=32)
builder.set_adam_optimizer(adam_params)

# Set the number of epochs
builder.set_epochs(100)

# Optionally, set descriptions for your training inputs
spec.description.trainingInput[0].shortDescription = 'Input data'
spec.description.trainingInput[1].shortDescription = 'Target output data'

spec.description.output[0].name = 'output_prob'
spec.description.output[0].shortDescription = 'Probability distribution over output classes'

# Save the updated model
updated_model = coremltools.models.MLModel(spec)
updated_model.save('updatable_classification.mlmodel')


In [2]:
builder.spec.description.output

[name: "output_prob"
shortDescription: "Probability distribution over output classes"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]

In [3]:
print(builder.spec.neuralNetwork)

layers {
  name: "linear_0"
  input: "x"
  output: "linear_0"
  isUpdatable: true
  innerProduct {
    inputChannels: 3
    outputChannels: 100
    hasBias: true
    weights {
      floatValue: 0.3256458
      floatValue: 0.24190164
      floatValue: -0.37449187
      floatValue: 0.20169449
      floatValue: 0.014378905
      floatValue: -0.0021227598
      floatValue: -0.24283808
      floatValue: 0.04516101
      floatValue: -0.31663963
      floatValue: 0.5661286
      floatValue: 0.37231624
      floatValue: -0.29687184
      floatValue: 0.3901806
      floatValue: 0.16699237
      floatValue: 0.3044067
      floatValue: -0.49332654
      floatValue: -0.5229649
      floatValue: -0.4719899
      floatValue: -0.34196803
      floatValue: -0.060092032
      floatValue: 0.12668914
      floatValue: 0.19274992
      floatValue: 0.1888215
      floatValue: -0.38902742
      floatValue: 0.32209384
      floatValue: 0.33408612
      floatValue: -0.037695885
      floatValue: 0.08493662
  